트랜스포머는 RNN 계열의 seq2seq를 대체하기 위해 등장함. 따라서 트랜스포머의 인코더는 RNN 인코더, 트랜스포머의 디코더는 RNN 디코더로 대체 가능

트랜스포머의 인코더는 셀프 어텐션이라는 메커니즘을 통해 문장을 이해함. RNN과 동작 방식은 다르지만, RNN이 텍스트 분류나 개체명 인식과 같은 다양한 자연어 처리 태스크에 쓰일 수 있음


실제, 트랜스포머의 인코더는 다양한 분야의 자연어 처리에서 사용될 수 있으며, 이 아이디어는 BERT 모델로 이어지게 됨. 여기에서는 트랜스포머의 인코더를 이용하여 텍스트분류를 수행해 봄

### 1. 멀티헤드 어텐션
------------------
트랜스포머의 인코더 첫번째 서브층인 멀티헤드 어텐션층을 클래스로 구현함

In [1]:
import tensorflow as tf

In [2]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim  # d_model
        self.num_heads = num_heads
        
        assert embedding_dim % self.num_heads == 0
        
        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)
        
    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights
    
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        
        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        
        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)
        
        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        
        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

### 2. 인코더 설계
-----------------
멀티 헤드 어텐션에 두번째 서브층인 포지션 와이즈 포워드 신경망을 추가하여 인코더 클래스 설계


In [3]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation='relu'), 
             tf.keras.layers.Dense(embedding_dim)]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        
    def call(self, inputs, training):
        attn_output = self.att(inputs)  # 첫번째 서브층 : 멀티헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add + Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이드 FFN
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add + Norm

### 3. 포지션 임베딩
------------------
앞서 트랜스포머에서는 포지셔널 인코딩을 사용하였지만, 이번에는 위치정보 자체를 학습하도록 하는 포지션 임베딩이라는 방법 사용

포지션 임베딩은 임베딩층(Embedding layer)을 사용하되, 위치 벡터를 학습하므로 임베딩 층의 첫번째 인자로 단어집합의 크기가 아니라 문장의 최대 길이를 넣어줌

In [4]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.pos_emb = tf.keras.layers.Embedding(max_len, embedding_dim)
        
    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

### 4. 데이터 로드 및 전처리
----------------------------

In [5]:
vocab_size = 20000  # 빈도수 상위 2만개 단어만 사용
max_len = 200 # 문장 최대 길이

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수: {}'.format(X_train.shape[0]))
print('테스트용 리뷰 개수 : {}'.format(X_test.shape[0]))

훈련용 리뷰 개수: 25000
테스트용 리뷰 개수 : 25000


In [7]:
print(X_train[0], len(X_train[0]))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32] 218


In [8]:
# 패딩
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

### 5. 트랜스포머를 이용한 IMDB 리뷰 분류
---------------------------------------

In [11]:
embedding_dim = 32 # 각 단어의 임베딩 벡터의 차원
num_heads = 2 
dff = 32 # 포지션 와이즈 FNN 은닉층 크기

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)

transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [12]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=32, epochs=2, validation_data=(X_test, y_test))

Epoch 1/2
782/782 [==============================] - 35s 45ms/step - loss: 0.3739 - accuracy: 0.8214 - val_loss: 0.3094 - val_accuracy: 0.8670
Epoch 2/2
782/782 [==============================] - 35s 45ms/step - loss: 0.1968 - accuracy: 0.9248 - val_loss: 0.3289 - val_accuracy: 0.8628


In [13]:
print("테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 10s 13ms/step - loss: 0.3289 - accuracy: 0.8628
테스트 정확도: 0.8628
